In [ ]:
import numpy as np
from levitt_model.model import LevittModel
from levitt_model.potential import Potential
import Bio.PDB as pdb
from Bio.PDB.QCPSuperimposer import QCPSuperimposer

import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Часть 0

## 0.1. Напишите <a href="https://t.me/protein_course_bot">этому боту в Telegram</a>, он вернет ID 10 тестовых белков. Загрузите первый из них и откройте его:

In [ ]:
pdb_ids = # TODO

In [ ]:
pdb_list = pdb.PDBList()

In [ ]:
path = pdb_list.retrieve_pdb_file(pdb_ids[0], file_format="pdb")

In [ ]:
struct = pdb.PDBParser().get_structure("s", path)

In [ ]:
chain = struct[0].child_list[0].child_list

## 0.2. Проверим применимость модели Левитта: все углы $\omega$ должны быть около $180$ грудусов. Если получится, что это не так для взятого белка, возьмите другой.

In [ ]:
omegas = []

for i in range(1, len(chain)):
    # your code here

omegas = np.array(omegas)
omegas[omegas < 0] += 2*np.pi

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(omegas/np.pi*180, bins=50)
plt.show()

# Часть 1

## 1.1. Давайте посмотрим на точность модели. Получите список названий аминокислот из цепочки; получите все оригинальные углы $\alpha_i = dihedral(C_\alpha^{i-1}, C_\alpha^{i}, C_\alpha^{i+1}, C_\alpha^{i+2})$

In [ ]:
res_names = [residue.resname for residue in chain]
orig_alphas = []

# your code here

## 1.2. Получите оригинальные координаты всех атомов $C_\alpha$; получите координаты $C_\alpha$, воспользовавшись моделью Левитта; сравните и получите точность.

In [ ]:
orig_cas = np.array([residue["CA"].coord for residue in chain])

model = LevittModel('table1.csv', res_names)
pred_cas = model.dihedralsToCoords(orig_alphas)[0]

In [ ]:
qsi = QCPSuperimposer()
qsi.set(orig_cas, pred_cas)
qsi.run()
print("Точность построенной модели: {}".format(qsi.rms))

Хороша ли модель? 🤔

# Часть 2

In [ ]:
potential = Potential(res_names, model, 'vdwCoeffs.csv', 'solventCoeffs.csv', 'fourierCoeffs.csv')

## 2.1. Вычислите значения потенциала Левитта для оригинальных значений углов $\alpha$:

In [ ]:
# your code here

**Для обсуждения**: Хорошее ли значение получилось? 🤔

### Проанализируем это значение. Вызовите ту же фунцию с параметром verbose=True:

In [ ]:
# your code here

Посмотрите на формулу потенциала. Как бы Вы объяснили такой результат?

## 2.2. Посмотрим на значения потенциала в других точках. Прежде всего, проверьте, каковы значения потенциала в 'полностью растянутой' конформации (все $\alpha = 180$) и 'полностью сжатой' (все $\alpha = 0$). Вызывайте функцию с параметром verbose=True.

Все 0:

In [ ]:
# your code here

Все 180:

In [ ]:
# your code here

Какие выводы можно сделать из этих результатов?

## 2.3. Запустим минимизацию из оригинальных значений $\alpha$. Каким будет результат?

In [ ]:
potential._init_alphas = orig_alphas
minimize(potential.calculateEnergyPotential,
           method="L-BFGS-B",
         jac=potential.calculate_potential_alpha_derivative,
           x0=all_zeros,
           bounds=[[-np.pi, np.pi]] * len(orig_alphas))